In [1]:
import pandas as pd
import numpy as np

from tensorflow import keras

from keras import models
from keras import layers

from sklearn.metrics import r2_score

from sklearn.preprocessing import MinMaxScaler
from scipy.stats.mstats import winsorize

from sklearn.model_selection import ParameterGrid

from tqdm import tqdm

from keras import regularizers

from keras.callbacks import EarlyStopping

In [2]:
#load data --> winsorize data --> scale data

df = pd.read_csv ('/Users/wvh/Documents/THESIS/Dataset_unscaled.csv', index_col=0)
df.sort_values(by=['year','month'], inplace=True)
df = df.reset_index(drop=True)

df.iloc[:,16:] = df.iloc[:,16:].apply(lambda x : winsorize(x,(0.1,0.1)))

cols_to_scale = list(df.iloc[:,15:].columns)
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

df[cols_to_scale] = min_max_scaler.fit_transform(df[cols_to_scale])

In [3]:
#create tuning parameters

# dropout%
dropout = [0.2,0.4]
# L1 penalty
L1_penalty = [0,0.001,0.00001]
# L2 penalty
L2_penalty = [0,0.001,0.00001]   
# learning rate
LR = [0.001,0.01]

param_grid = {'dropout': dropout,'L1_penalty': L1_penalty,'L2_penalty': L2_penalty, 'LR': LR}
tuning_parameters = ParameterGrid(param_grid)

In [5]:
seed_value= 42

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

tf.compat.v1.keras.backend.set_session(sess)

tf.keras.utils.set_random_seed(seed_value)

In [6]:
R2_list_NN_2 = []

for name in df.iloc[:,9:15]:
    
    df_iter = df[df[name] <= df[name].quantile(0.95)].copy()
    
    for p in range(1,6):
    
        years = np.arange(1996, 2013+p, 1)

        locals()['df_train_' + str(p)] = df_iter[df_iter.year.isin(years)]

        year = [2013 + p,2014 + p,2015 + p]

        locals()['df_val_' + str(p)] = df_iter[df_iter.year.isin(year)]

        year = [2016 + p]

        locals()['df_test_' + str(p)] = df_iter[df_iter.year.isin(year)]
        
    true = []
    predicted = []
    
    for j in range(1,6):
        y_train = globals()['df_train_'+str(j)][name]
        y_val = globals()['df_val_'+str(j)][name]
        y_test = globals()['df_test_'+str(j)][name]

        X_train = globals()['df_train_'+str(j)].iloc[:,16:]
        X_val = globals()['df_val_'+str(j)].iloc[:,16:]
        X_test = globals()['df_test_'+str(j)].iloc[:,16:]

        val_r2_list = []
        optimal_hyperparams = []

        for i in tqdm(range(len(tuning_parameters))):

            es = EarlyStopping(monitor='val_loss', verbose=0, patience=10, restore_best_weights=True,mode='min')
            
            def model_1():
                model = models.Sequential()
                model.add(layers.Dense(64, activation='relu',
                                      input_shape = (X_train.shape[1],),kernel_regularizer=regularizers.l1_l2(l1 = tuning_parameters[i]['L1_penalty'],l2 = tuning_parameters[i]['L2_penalty'])))
                model.add(layers.Dropout(tuning_parameters[i]['dropout']))
                model.add(layers.Dense(32, activation = 'relu',kernel_regularizer=regularizers.l1_l2(l1 = tuning_parameters[i]['L1_penalty'],l2 = tuning_parameters[i]['L2_penalty'])))
                model.add(layers.Dropout(tuning_parameters[i]['dropout']))
                model.add(layers.Dense(1))

                opti = keras.optimizers.Adam(learning_rate=tuning_parameters[i]['LR'])

                model.compile(optimizer=opti,loss='mse',metrics=['mae'])
                return model

            # fit the regressor with x and y data
            NN = model_1()
            NN.fit(X_train,y_train,epochs=1000,batch_size=512*16,verbose=0,callbacks=[es],validation_data=(X_val, y_val))

            y_pred_val = NN.predict(X_val)

            val_r2_list.append(r2_score(y_val, y_pred_val)*100)

            opt = tuning_parameters[np.argmax(val_r2_list)]

        print('The optimal tuning parameters are:')
        print(name,j,opt)
        optimal_hyperparams.append([opt])
        
        es = EarlyStopping(monitor='val_loss', verbose=0, patience=10, restore_best_weights=True,mode='min')

        def model_1():
            model = models.Sequential()
            model.add(layers.Dense(64, activation='relu',
                                  input_shape = (X_train.shape[1],),kernel_regularizer=regularizers.l1_l2(l1 = opt['L1_penalty'],l2 = opt['L2_penalty'])))
            model.add(layers.Dropout(opt['dropout']))
            model.add(layers.Dense(32, activation = 'relu',kernel_regularizer=regularizers.l1_l2(l1 = opt['L1_penalty'],l2 = opt['L2_penalty'])))
            model.add(layers.Dropout(opt['dropout']))
            model.add(layers.Dense(1))

            opti = keras.optimizers.Adam(learning_rate=opt['LR'])

            model.compile(optimizer=opti,loss='mse',metrics=['mae'])
            return model
    
        NN = model_1()
        NN.fit(X_train,y_train,epochs=1000,batch_size=512*16,verbose=0,callbacks=[es],validation_data=(X_val, y_val)) 

        y_pred_test = NN.predict(X_test)
        
        true.extend(y_test)

        predicted.extend(y_pred_test)
        
    R2_list_NN_2.append([name,r2_score(true, predicted)*100])
    print([name,r2_score(true, predicted)*100])

print()
print('THESE ARE THE FINAL R2 FOR NN2')
print(R2_list_NN_2)

  0%|                                                    | 0/36 [00:00<?, ?it/s]

433/433 [==============================] - 0s 809us/step


  3%|█▏                                          | 1/36 [00:23<13:30, 23.15s/it]

433/433 [==============================] - 0s 721us/step


  6%|██▍                                         | 2/36 [00:55<16:01, 28.28s/it]

...

 97%|█████████████████████████████████████████▊ | 35/36 [27:29<01:06, 66.96s/it]

428/428 [==============================] - 0s 591us/step


100%|███████████████████████████████████████████| 36/36 [28:22<00:00, 47.29s/it]

The optimal tuning parameters are:
IV_D_P_91 5 {'dropout': 0.2, 'LR': 0.01, 'L2_penalty': 1e-05, 'L1_penalty': 1e-05}


13/13 [==============================] - 0s 684us/step
['IV_D_P_91', 12.621668972261501]

THESE ARE THE FINAL R2 FOR NN2
[['IV_D_C_30', 9.709736139354241], ['IV_D_P_30', 10.451960493423229], ['IV_D_C_60', 10.756290632431764], ['IV_D_P_60', 11.650961691619088], ['IV_D_C_91', 11.57712532921067], ['IV_D_P_91', 12.621668972261501]]
